In [ ]:
!pip install open3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.7 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: flask
    Found existing installation: Flask 3.1.0
    Uninst

In [ ]:
!pip install open3d matplotlib numpy


In [ ]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

In [ ]:
import sqlite3

In [ ]:
import numpy as np

txt_file = 'lidar_output.txt'

# Read each line, split into x/y/z, and convert to float
points_list = []
with open(txt_file, 'r') as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        x_str, y_str, z_str = line.split()
        points_list.append([float(x_str), float(y_str), float(z_str)])

points_np = np.array(points_list, dtype=np.float64)
print("Loaded points:", points_np.shape)

Loaded points: (32305, 3)


In [ ]:
import open3d as o3d

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_np)
print("Point cloud size:", len(pcd.points))

Point cloud size: 32305


In [ ]:
threshold = 1e6
mask = (
    (np.abs(points_np[:, 0]) < threshold) &
    (np.abs(points_np[:, 1]) < threshold) &
    (np.abs(points_np[:, 2]) < threshold)
)
filtered_points_np = points_np[mask]
print("Points after filtering:", filtered_points_np.shape)

pcd_filtered = o3d.geometry.PointCloud()
pcd_filtered.points = o3d.utility.Vector3dVector(filtered_points_np)

Points after filtering: (23373, 3)


In [ ]:
# Downsample
voxel_size = 0.1
pcd_down = pcd_filtered.voxel_down_sample(voxel_size=voxel_size)

# Statistical Outlier Removal
num_neighbors = 20
std_ratio = 2.0
pcd_clean, ind = pcd_down.remove_statistical_outlier(
    nb_neighbors=num_neighbors,
    std_ratio=std_ratio
)
print("Points after cleaning:", len(pcd_clean.points))


Points after cleaning: 5383


In [ ]:
distance_threshold = 0.2
ransac_n = 3
num_iterations = 1000

plane_model, inliers = pcd_clean.segment_plane(
    distance_threshold=distance_threshold,
    ransac_n=ransac_n,
    num_iterations=num_iterations
)
[a, b, c, d] = plane_model
print(f"Plane equation: {a}x + {b}y + {c}z + {d} = 0")

inlier_cloud = pcd_clean.select_by_index(inliers)          # Ground plane points
outlier_cloud = pcd_clean.select_by_index(inliers, invert=True)  # Objects above ground


inlier_cloud.paint_uniform_color([0.0, 1.0, 0.0])


Plane equation: -7.052603977135891e-09x + 6.9042236048396e-10y + 1.0z + -0.0038487565658877083 = 0


PointCloud with 1566 points.

In [ ]:
import matplotlib.pyplot as plt

eps = 0.5       # max distance between points in a cluster
min_points = 5  # minimum points to form a cluster

with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
    labels = np.array(outlier_cloud.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))

max_label = labels.max()
print(f"Detected {max_label + 1} clusters")

#colors to each cluster
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0  # noise or unclustered
outlier_cloud.colors = o3d.utility.Vector3dVector(colors[:, :3])


Detected 105 clusters


In [ ]:
bounding_boxes = []
for cluster_id in range(max_label + 1):
    cluster_indices = np.where(labels == cluster_id)[0]
    cluster_cloud = outlier_cloud.select_by_index(cluster_indices)

    aabb = cluster_cloud.get_axis_aligned_bounding_box()
    aabb.color = (1, 0, 0)  # red
    bounding_boxes.append(aabb)


In [ ]:
bounding_boxes

[AxisAlignedBoundingBox: min: (-1.513, 0.725, 80), max: (-1.083, 1.484, 80),
 AxisAlignedBoundingBox: min: (-4.197, -1.7905, -0.407), max: (3.904, 1.552, 2.112),
 AxisAlignedBoundingBox: min: (-1.668, -0.101, 82), max: (-1.128, 1.144, 82),
 AxisAlignedBoundingBox: min: (-1.523, 0.709, 79), max: (-1.073, 1.465, 79),
 AxisAlignedBoundingBox: min: (-1.653, 0.139, 88), max: (-1.112, 0.947, 88),
 AxisAlignedBoundingBox: min: (-1.627, -0.082, 13), max: (-1.2375, 0.885, 13),
 AxisAlignedBoundingBox: min: (-1.442, 1.07, 71), max: (-0.995, 1.76, 71),
 AxisAlignedBoundingBox: min: (368.266, -5.489, -2.047), max: (368.266, 6.35, -0.211333),
 AxisAlignedBoundingBox: min: (-0.082, 0, 1.837), max: (0.3735, 0.429, 2),
 AxisAlignedBoundingBox: min: (368.266, -6.319, 0.29275), max: (368.266, 4.496, 1.725),
 AxisAlignedBoundingBox: min: (1, 1.192, 49), max: (1.375, 1.446, 49),
 AxisAlignedBoundingBox: min: (1.011, 0.437, 11), max: (1.13967, 0.976333, 11),
 AxisAlignedBoundingBox: min: (0.873, -0.055, 13